In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [87]:
import re
from datetime import datetime
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [88]:
locator = Nominatim(user_agent="ppcoom")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1/20)
dengue_regex = re.compile(r'([A-Za-z ]+).*\[w\/e (.+)\] \/ (.+) \/ (.+) \/ (.+) \/ (.+) \/ (.+)', re.MULTILINE)

In [4]:
df = pd.read_feather('combined_df_anomaly.feather')

In [5]:
dengue_df = df[df['disease'] == 'dengue']

In [117]:
def parse_dengue(row):
    return pd.DataFrame([(
        *row[2:],
        match[0].strip(),
        pd.NA if re.match(r'\d+ \w{3} \w{4}', re.sub(r'[^\w ]+', '', match[1])) is None 
            else datetime.strptime(re.sub(r'[^\w ]+', '', match[1]), r'%d %b %Y'),
        match[2],
        *[pd.NA if not match[i].isnumeric() else int(match[i].replace(' ', '')) for i in range(3, 7)])
        for match in dengue_regex.findall(row['content'])],
        columns=[
        *row[2:].keys(),
        'location_name',
        'dates_start',
        'serotype',
        'total_cases',
        'confirmed_cases',
        'severe_cases',
        'deaths'
    ])


In [118]:
dengue_df_parse = pd.concat([parse_dengue(row) for i, row in tqdm(dengue_df.iterrows())])


1412it [00:46, 30.23it/s] 


In [120]:
dengue_df_parse['location'] = dengue_df_parse['location_name'].progress_apply(geocode)
dengue_df_parse['point'] = dengue_df_parse['location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)
dengue_df_parse[['location_lat', 'location_lon', 'altitude']] = pd.DataFrame(dengue_df_parse['point'].tolist(), index=dengue_df_parse.index)
dengue_df_parse = dengue_df_parse.drop(['location', 'point', 'altitude'], axis=1)
dengue_df_parse = dengue_df_parse.reset_index(drop=True)

100%|██████████| 788/788 [00:00<00:00, 196943.84it/s]


KeyError: "['location' 'point' 'altitude'] not found in axis"

In [157]:
dengue_df_parse.to_feather('dengue_df.feather')

In [159]:
dengue_df = pd.read_feather('dengue_df.feather')
other_df = pd.read_feather('dataset.v1.1.feather')

In [168]:
dengue_df = dengue_df.rename({'confirmed_cases': 'cases'}, axis=1)
full_df = pd.concat([dengue_df,other_df], axis=0, ignore_index=True)

In [169]:
len(full_df)

2112

In [172]:
full_df = full_df.drop(['Unnamed: 0', 'index'], axis=1)

In [173]:
full_df.to_feather('dataset.v1.2.feather')

In [171]:
full_df.iloc[0]

id                                                                 8163513
title                                 ['Dengue/DHF update (01): Americas']
zoom_lat                                                          23.95046
zoom_lon                                                        -102.53289
zoom_level                                                               5
alert_id                                                           8163513
feed_id                                                                  1
summary                          PRO/EDR> Dengue/DHF update (01): Americas
issue_date                                             2021-02-04 06:39:21
load_date                                              2021-02-04 06:39:21
incident_date                                                          NaN
descr                    Cases by Country / Week updated / Serotype / T...
alert_tag_id                                                           NaN
dup_count                